In [105]:
import sys
sys.path.append("..")
sys.path.append("../..")
sys.path.append("../../..")

from py.browser_parser import CommentParser, PostParser
from py.crawler import PostCrawler
from py.config import *

import random
import pandas as pd
import os
import datetime
import time
import json

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from selenium.webdriver.chrome.service import Service


In [107]:
options = webdriver.ChromeOptions()  # configure the webdriver
options.add_argument('no-sandbox')
options.add_argument('disable-dev-shm-usage')
options.add_argument('lang=zh_CN.UTF-8')
options.add_argument('user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, '
                        'like Gecko) Chrome/111.0.0.0 Safari/537.36"')
options.add_argument('headless')
browser = webdriver.Chrome(options=options, service=Service("./resource\chromedriver\win64/125.0.6422.60\chromedriver.exe"))

In [108]:
pp = PostParser()

In [109]:
url1 = "https://guba.eastmoney.com/news,000005,1428298677.html"
url2 = "https://guba.eastmoney.com/news,000001,1428341745.html"

In [115]:
browser.get(url1)
x = PostParser().get_post_text(browser)
x


'世纪星原 00000005，坐等'

In [113]:
import threading

def f(s, issubclass):
    with s:
        browser.get(url2)
        print(f"End {i}")
        x = PostParser().get_post_text(browser)

In [114]:
s = threading.Semaphore(100)
for i in range(10000):
    t = threading.Thread(target=f, args=(s, i))
    t.start()
    t.join()
print("all finish")

End 0
End 1
End 2
End 3
End 4
End 5
End 6
End 7
End 8
End 9
End 10
End 11
End 12
End 13
End 14
End 15
End 16
End 17
End 18
End 19
End 20
End 21
End 22
End 23
End 24
End 25
End 26
End 27
End 28
End 29
End 30
End 31
End 32
End 33
End 34
End 35
End 36
End 37
End 38
End 39
End 40
End 41
End 42
End 43
End 44
End 45
End 46
End 47
End 48
End 49
End 50
End 51
End 52
End 53
End 54
End 55
End 56
End 57
End 58
End 59
End 60
End 61
End 62
End 63
End 64
End 65
End 66
End 67
End 68
End 69
End 70
End 71
End 72
End 73
End 74
End 75
End 76
End 77
End 78
End 79
End 80
End 81
End 82
End 83
End 84
End 85
End 86
End 87
End 88
End 89
End 90
End 91
End 92
End 93
End 94
End 95
End 96
End 97
End 98
End 99
End 100
End 101
End 102
End 103
End 104
End 105
End 106
End 107
End 108
End 109
End 110
End 111
End 112
End 113
End 114
End 115
End 116
End 117
End 118
End 119
End 120
End 121
End 122
End 123
End 124
End 125
End 126
End 127
End 128
End 129
End 130
End 131
End 132
End 133
End 134
End 135
End 136
End 137
End 13

KeyboardInterrupt: 

In [88]:
x

'以下转自网易财经9月23日，由网易财经主办的“2023网易财经ESG趋势论坛”在深圳举行。本届论坛主题是“凝聚ESG共识、聚焦关键行动”，由广州市绿色金融协会、深圳市南山区金融行业协会、大湾区碳中和协会提供支持。来自ESG国际组织、评级机构、学界、企业界的嘉宾出席了本次论坛，并基于ESG议题发表重要演讲与深入交流。平安银行绿色金融事业部总裁朱育强在论坛上圆桌对话环节表示，从国际和国内来看，绿色已经成为中国经济高质量发展的鲜明底色和核心要义。据平安银行测算，预计我国到2050年实现碳中和路径至少还需要130万亿元的投融资规模，需要绿色金融强有力的支持。朱育强同时提出，对于金融机构而言，绿色金融要扮演好四个重要角色，即区域经济转型的“推进器”、企业绿色发展的“好伙伴”、个人绿色消费的“引导者”、国际竞争合作的“护航队”。近年来，平安银行在绿色金融实践中形成了“两新一稳”的亮点，即新型的组织体系、创新的产品模式与稳健的风控逻辑。'

In [92]:
info = json.loads(browser.execute_script("return JSON.stringify(post_article)"))

In [93]:
x = etree.HTML(info['post_content'])
"".join(x.xpath('//*[@class="xeditor_content editorlungo_content"]/p/text()'))

'以下转自财经9月23日，由网易财经主办的“2023网易财经ESG趋势论坛”在深圳举行。本届论坛主题是“凝聚ESG共识、聚焦关键行动”，由广州市绿色金融协会、深圳市南山区金融行业协会、大湾区碳中和协会提供支持。来自ESG国际组织、评级机构、学界、企业界的嘉宾出席了本次论坛，并基于ESG议题发表重要演讲与深入交流。绿色金融事业部总裁朱育强在论坛上圆桌对话环节表示，从国际和国内来看，绿色已经成为中国经济高质量发展的鲜明底色和核心要义。据平安银行测算，预计我国到2050年实现碳中和路径至少还需要130万亿元的投融资规模，需要绿色金融强有力的支持。朱育强同时提出，对于金融机构而言，绿色金融要扮演好四个重要角色，即区域经济转型的“推进器”、企业绿色发展的“好伙伴”、个人绿色消费的“引导者”、国际竞争合作的“护航队”。近年来，平安银行在绿色金融实践中形成了“两新一稳”的亮点，即新型的组织体系、创新的产品模式与稳健的风控逻辑。'

In [56]:
info['post_title']
info['user_age']
info['user_age']
info['user_influ_level']
info['user_age']
info['user_age']

'平安银行朱育强：绿色金融要扮演好四个重要角色'

['以下转自',
 '财经',
 '9月23日，由网易财经主办的“2023网易财经ESG趋势论坛”在深圳举行。本届论坛主题是“凝聚ESG共识、聚焦关键行动”，由广州市绿色金融协会、深圳市南山区金融行业协会、大湾区碳中和协会提供支持。来自ESG国际组织、评级机构、学界、企业界的嘉宾出席了本次论坛，并基于ESG议题发表重要演讲与深入交流。',
 '绿色金融事业部总裁朱育强在论坛上圆桌对话环节表示，从国际和国内来看，绿色已经成为中国经济高质量发展的鲜明底色和核心要义。据平安银行测算，预计我国到2050年实现碳中和路径至少还需要130万亿元的投融资规模，需要绿色金融强有力的支持。',
 '朱育强同时提出，对于金融机构而言，绿色金融要扮演好四个重要角色，即区域经济转型的“推进器”、企业绿色发展的“好伙伴”、个人绿色消费的“引导者”、国际竞争合作的“护航队”。近年来，平安银行在绿色金融实践中形成了“两新一稳”的亮点，即新型的组织体系、创新的产品模式与稳健的风控逻辑。']

In [62]:
print(info['post_content'])

<div class="xeditor_content editorlungo_content"><p>以下转自<a href="https://quote.eastmoney.com/unify/r/105.NTES">网易</a>财经</p><p><br></p><p></p><p></p><p>9月23日，由网易财经主办的“2023网易财经ESG趋势论坛”在深圳举行。本届论坛主题是“凝聚ESG共识、聚焦关键行动”，由广州市绿色金融协会、深圳市南山区金融行业协会、大湾区碳中和协会提供支持。来自ESG国际组织、评级机构、学界、企业界的嘉宾出席了本次论坛，并基于ESG议题发表重要演讲与深入交流。</p><p></p><p></p><p><a href="https://quote.eastmoney.com/unify/r/0.000001">平安银行</a>绿色金融事业部总裁朱育强在论坛上圆桌对话环节表示，从国际和国内来看，绿色已经成为中国经济高质量发展的鲜明底色和核心要义。据平安银行测算，预计我国到2050年实现碳中和路径至少还需要130万亿元的投融资规模，需要绿色金融强有力的支持。</p><p></p><p></p><p>朱育强同时提出，对于金融机构而言，绿色金融要扮演好四个重要角色，即区域经济转型的“推进器”、企业绿色发展的“好伙伴”、个人绿色消费的“引导者”、国际竞争合作的“护航队”。近年来，平安银行在绿色金融实践中形成了“两新一稳”的亮点，即新型的组织体系、创新的产品模式与稳健的风控逻辑。</p><p></p><br><p></p><p><br></p></div>


In [24]:
import requests
from lxml import etree

headers= {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7", 
        "Accept-Encoding": "gzip, deflate, br, zstd",
        "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8",
        "Connection": "keep-alive",
        "Cookie": "channelid=0; sid=1715219981038637; _gid=GA1.2.248850560.1715220633; __51vcke__K3h4gFH3WOf3aJqX=bdda2b15-b174-5aec-9a62-0c5f259351cf; __51vuft__K3h4gFH3WOf3aJqX=1715220633183; _gcl_au=1.1.1224160315.1715220633; _ss_s_uid=d72830c0818227cfd3b3f8b27b92ab96; sessionid=ec8c0c1a8faf5642e57d1003c1ea353b; Hm_lvt_7ed65b1cc4b810e9fd37959c9bb51b31=1715221103; Hm_lpvt_7ed65b1cc4b810e9fd37959c9bb51b31=1715221103; __51uvsct__K3h4gFH3WOf3aJqX=2; _ga_DC1XM0P4JL=GS1.1.1715244313.2.1.1715244330.43.0.0; _ga=GA1.1.1580103044.1715220633; __vtins__K3h4gFH3WOf3aJqX=%7B%22sid%22%3A%20%2215171645-4cf6-5800-b249-34ec77f040bc%22%2C%20%22vd%22%3A%207%2C%20%22stt%22%3A%2017371%2C%20%22dr%22%3A%203300%2C%20%22expires%22%3A%201715246130271%2C%20%22ct%22%3A%201715244330271%7D",
        "User-Agent": "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 10.0; WOW64; Trident/7.0; .NET4.0C; .NET4.0E; Tablet PC 2.0; wbx 1.0.0; wbxapp 1.0.0; Zoom 3.6.0)", 
        # "User-Agent": UserAgent().random
    }

In [71]:
from tqdm import trange

In [78]:
for i in trange(10000):
    r = requests.get("https://guba.eastmoney.com/news,000001,1428341745.html", headers=headers)


  1%|          | 64/10000 [00:10<28:25,  5.83it/s]


KeyboardInterrupt: 

In [79]:
r.encoding = r.apparent_encoding
print(r.text)


'utf-8'

<!DOCTYPE html><html><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"/><meta name="renderer" content="webkit"/><meta charSet="utf-8"/><base target="_blank"/><link rel="stylesheet" href="//gbfek.dfcfw.com/deploy/fd_guba_web2022/work/error404.css"/><body class="hlbody"><div id="pageContent" class="error404_page"><div id="menu"></div><div id="top"><div id="search"><div class="skeleton"><div class="container h100"><div class="square w40 h60"></div></div></div></div></div><div id="fix-head"></div><div id="middle" class="cl"></div><div id="bottom"></div></div><div id="footerHtml"><div><!-- footer-2016 -->
<style>
    .footer2016 ul {list-style: none;margin: 0;padding: 0;}
    .footer2016 a:hover{color:#ff4901 !important;text-decoration: underline !important;}
    .footer2016 .icon, .footer2016 .navlist li a {display: inline-block;}
    .footer2016 .icon {background-image: url(//g1.dfcfw.com/g4/202005/20200513165332.png);background-repeat: no-repeat;}
    .footer2016 {margin: 0 a

In [38]:
""r.text
t = etree.HTML(r.text) 

In [ ]:
//*[@id="newscontent"]/div[4]

In [49]:
t.xpath('//*/script')

[<Element script at 0x1a09e58d500>,
 <Element script at 0x1a09e58dc40>,
 <Element script at 0x1a09e675ac0>,
 <Element script at 0x1a09e58d940>,
 <Element script at 0x1a09e677140>,
 <Element script at 0x1a09e5e8780>,
 <Element script at 0x1a09e5ea300>,
 <Element script at 0x1a09e5e9bc0>,
 <Element script at 0x1a09e5eb2c0>,
 <Element script at 0x1a09e5e8ec0>,
 <Element script at 0x1a09e5eb3c0>,
 <Element script at 0x1a09e5e97c0>,
 <Element script at 0x1a09e5e85c0>,
 <Element script at 0x1a09e5e8540>,
 <Element script at 0x1a09e5413c0>,
 <Element script at 0x1a09e5eb9c0>,
 <Element script at 0x1a09e5eacc0>,
 <Element script at 0x1a09e5ea140>,
 <Element script at 0x1a09e5ea900>,
 <Element script at 0x1a09e5e8b80>,
 <Element script at 0x1a09e5eac00>]